In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 8]
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import pickle

In [11]:
raw_data = pd.read_csv('datasets/samsung200129_220328.csv')
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 534 entries, 0 to 533
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       534 non-null    object 
 1   Open       534 non-null    float64
 2   High       534 non-null    float64
 3   Low        534 non-null    float64
 4   Close      534 non-null    float64
 5   Adj Close  534 non-null    float64
 6   Volume     534 non-null    int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 29.3+ KB


In [12]:
raw_data['Date'] = pd.to_datetime(raw_data['Date'])
raw_data.set_index('Date', inplace=True)
print(raw_data.head())

               Open     High      Low    Close     Adj Close    Volume
Date                                                                  
2020-01-29  59100.0  59700.0  58800.0  59100.0  55485.578125  16446102
2020-01-30  58800.0  58800.0  56800.0  57200.0  53701.785156  20821939
2020-01-31  57800.0  58400.0  56400.0  56400.0  52950.710938  19749457
2020-02-03  55500.0  57400.0  55200.0  57200.0  53701.785156  23995260
2020-02-04  57100.0  59000.0  56800.0  58900.0  55297.820313  21800192


In [13]:
minmaxscaler = MinMaxScaler()
scaled_data = minmaxscaler.fit_transform(raw_data)
print(scaled_data[:5])
print(scaled_data.shape)  # 이제 시퀀셜 데이터로 바꿔줘야합니다

[[0.34591195 0.30328638 0.34957627 0.34226804 0.31505435 0.18211492]
 [0.33962264 0.28638498 0.30720339 0.30309278 0.27899405 0.23057048]
 [0.31865828 0.27887324 0.29872881 0.28659794 0.26381069 0.21869442]
 [0.27044025 0.2600939  0.27330508 0.30309278 0.27899405 0.26571006]
 [0.30398323 0.29014085 0.30720339 0.33814433 0.31125872 0.24140311]]
(534, 6)


In [14]:
sequence_X = []
sequence_Y = []

for i in range(len(scaled_data)-30):
    x = scaled_data[i:i+30]
    y = scaled_data[i+30][3]
    sequence_X.append(x)
    sequence_Y.append(y)
    
sequence_X = np.array(sequence_X)
sequence_Y = np.array(sequence_Y)
print(sequence_X[0])  # 6개씩 들어있는 데이터가 1개
print(sequence_Y[0])  # 종가
print(sequence_X.shape)
print(sequence_Y.shape)

[[0.34591195 0.30328638 0.34957627 0.34226804 0.31505435 0.18211492]
 [0.33962264 0.28638498 0.30720339 0.30309278 0.27899405 0.23057048]
 [0.31865828 0.27887324 0.29872881 0.28659794 0.26381069 0.21869442]
 [0.27044025 0.2600939  0.27330508 0.30309278 0.27899405 0.26571006]
 [0.30398323 0.29014085 0.30720339 0.33814433 0.31125872 0.24140311]
 [0.36477987 0.31267606 0.35169492 0.35051546 0.32264614 0.2134756 ]
 [0.36687631 0.32957746 0.36864407 0.38350515 0.35301285 0.16308031]
 [0.38784067 0.3314554  0.36864407 0.36907216 0.33972739 0.18163202]
 [0.34800839 0.30516432 0.3559322  0.35463918 0.326442   0.14514091]
 [0.360587   0.32206573 0.36864407 0.35876289 0.33023786 0.12259661]
 [0.37106918 0.32206573 0.36864407 0.37113402 0.34162535 0.14289396]
 [0.38993711 0.33896714 0.38559322 0.37525773 0.34542113 0.20430247]
 [0.3836478  0.34460094 0.37923729 0.39793814 0.36629816 0.14701173]
 [0.39832285 0.34647887 0.40042373 0.39175258 0.36060449 0.09678846]
 [0.38155136 0.3258216  0.36864407

In [15]:
X_train, X_test, Y_train, Y_test = train_test_split(sequence_X, sequence_Y, test_size=0.2)
xy = X_train, X_test, Y_train, Y_test
np.save('datasets/samsung_preprocessed_30.npy', xy)

C:\anaconda3\envs\DL_exam_37_opencv\lib\site-packages\numpy\lib\npyio.py:528: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


In [16]:
with open('datasets/samsung_minmaxscaler.pickle', 'wb') as f:
    pickle.dump(minmaxscaler, f)

In [17]:
with open('datasets/samsung_minmaxscaler.pickle', 'rb') as f:
    minmaxscaler = pickle.load(f)

In [18]:
model = Sequential()
model.add(LSTM(50, input_shape=(30, 6), activation='tanh'))
model.add(Flatten())
model.add(Dense(1))
model.compile(loss='mse', optimizer='adam')
model.summary()

NotImplementedError: Cannot convert a symbolic Tensor (lstm_1/strided_slice:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=30)
fit_hist = model.fit(X_train, Y_train, batch_size=128, epochs=500,
                     callbacks=[early_stopping], verbose=1,
                     validation_data=(X_test,Y_test), shuffle=False)

In [ ]:
plt.plot(fit_hist.history['loss'][-450:], label='loss')
plt.plot(fit_hist.history['val_loss'][-450:], label='val_loss')
plt.legend()
plt.show()

In [ ]:
pred = model.predict(X_test)

In [ ]:
plt.plot(Y_test[:30], label='actual')
plt.plot(pred[:30], label='predict')
plt.legend()
plt.show()